In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
os.chdir("/home/zhzhou/GNN_E")
import numpy as np
import pandas as pd
import torch
from torch_geometric.loader import DataLoader
from math import ceil
from utils_models import *
from scripts.models import *
from GCN_models import *

/home/zhzhou/.conda/envs/GNN_E/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_path = 'data/processed/all_movies_labelled_13_single.csv'
data_path = '/home/dalai/GNN_E/data/processed/all_movies_labelled_13_single_balanced.csv'
df =  pd.read_csv(data_path, index_col=0)

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
df_train, df_test = split_train_test_vertically(df)
dataset_train = DatasetEmo(df_train, device=DEVICE, node_feat="symmetricwindow")
dataset_test = DatasetEmo(df_test, device = DEVICE, node_feat="symmetricwindow")

graphs_list_train = dataset_train.get_graphs_list()
graphs_list_test = dataset_test.get_graphs_list()


Movies in this df: [ 0  1  2  3  4  5  6  8 11 12 13]


Processing 13 32: 100%|██████████| 310/310 [00:01<00:00, 235.83timepoint/s]


Movies in this df: [ 7  9 10]


Processing 10 32: 100%|██████████| 231/231 [00:00<00:00, 260.52timepoint/s]


In [9]:
from torch_geometric.loader import DataLoader
from math import ceil
batch_size = 16
train_loader = DataLoader(graphs_list_train, batch_size=batch_size, num_workers=4, persistent_workers=True)
test_loader = DataLoader(graphs_list_test, batch_size=batch_size, num_workers=4, persistent_workers=True)

num_batches_train = ceil(len(graphs_list_train) / batch_size)
num_batches_test = ceil(len(graphs_list_test) / batch_size)

print(f"There are {len(graphs_list_train)} graphs in the train set.")
print(f"There are {len(graphs_list_test)} graphs in the test set.")
print(f"N batches in train: {num_batches_train}")
print(f"N batches in test: {num_batches_test}")

There are 84890 graphs in the train set.
There are 27820 graphs in the test set.
N batches in train: 5306
N batches in test: 1739


In [ ]:
from tqdm import tqdm
input_dim = 9 
hidden_dim1 = 64
hidden_dim2 = 32
output_dim = 13  # Number of classes
print(isinstance(SimpleGCN(9, 64,32, 13), nn.Module))
model = SimpleGCN(input_dim, hidden_dim1, hidden_dim2, output_dim).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.CrossEntropyLoss()
num_epochs = 1
# Training loop
torch.cuda.empty_cache()
model, results_dict = GCN_train(model=model, optimizer=optimizer, loss_fn=loss_fn, 
                                    train_loader=train_loader, test_loader=test_loader, device=DEVICE, num_epochs=10)


True


Epoch 1/10:   4%|▎         | 191/5306 [00:13<06:12, 13.73it/s, Batch Loss=2.25, out=tensor(0, device='cuda:0')]

In [17]:
file_name = 'GCNModel.pth'
torch.save(model.state_dict(), file_name)

In [ ]:
def test():
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct = 0  # To keep track of the number of correct predictions
    total = 0    # To keep track of the total number of samples

    # No need for gradients during testing
    with torch.no_grad():
        progress_bar = tqdm(loader_test, desc="Testing", leave=False)
        for i, batch in enumerate(progress_bar):
            batch = batch.to(DEVICE)

            # Forward pass
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = loss_fn(out, batch.y)

            # Update total loss
            total_loss += loss.item()

            # Get the predicted class (index of the max logit)
            _, predicted = torch.max(out, dim=1)

            # Update the number of correct predictions and total samples
            correct += (predicted == batch.y).sum().item()
            print(predicted)
            print(batch.y)
            total += batch.y.size(0)

            # Update the progress bar with loss
            progress_bar.set_postfix({"Test Loss": predicted})

    # Calculate the average loss and accuracy
    avg_loss = total_loss / len(loader_test)
    accuracy = correct / total * 100  # Accuracy as a percentage

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")

    return avg_loss, accuracy

test()

In [20]:
print(predicted)

NameError: name 'predicted' is not defined